In [1]:
import nest_asyncio
import asyncio
import datetime
from typing import List
from pydantic import BaseModel
import openai
import instructor
import aiohttp
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

nest_asyncio.apply()

class DateRange(BaseModel):
    start: datetime.date
    end: datetime.date

class MetaphorQuery(BaseModel):
    rewritten_query: str
    published_daterange: DateRange
    domains_allow_list: List[str]

    async def execute(self):
        url = "https://api.exa.ai/search"
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "x-api-key": "bb451e54-049b-4586-9e6e-9582e3a992ce",
        }
        payload = {
            "query": self.rewritten_query,
            "type": "keyword",
            "numResults": 3,
            "startPublishedDate": self.published_daterange.start.isoformat() + "T00:00:00.000Z",
            "endPublishedDate": self.published_daterange.end.isoformat() + "T00:00:00.000Z",
            "category": "news",
            "contents": {
                "text": True
            }
        }

        async with aiohttp.ClientSession() as session:
            async with session.post(url, headers=headers, json=payload) as response:
                return await response.json()

# Enables response_model in the openai client
client = instructor.from_openai(openai.OpenAI(api_key=api_key))

async def generate_query_instance():
    # Generate parameters using OpenAI
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You are a query understanding system for the Metaphor Systems search engine. Here are some tips: ...",
            },
            {"role": "user", "content": "Generate a MetaphorQuery instance for the topic 'hottest AI startups'."},
        ],
        response_model=MetaphorQuery,
    )
    
    # Extract the response content
    query_data = response
    
    # Parse the generated response to extract parameters
    # Assuming the response is structured correctly (this part may require some adjustments based on actual response format)
    # query_data = eval(response_content)  # This is just a placeholder; adjust as needed
    
    # Construct the MetaphorQuery instance
    query_instance = MetaphorQuery(
        rewritten_query=query_data.rewritten_query,
        published_daterange=DateRange(
            start=query_data.published_daterange.start,
            end=query_data.published_daterange.end
        ),
        domains_allow_list=query_data.domains_allow_list
    )
    
    return query_instance

# Example function to demonstrate the execution
async def main():
    # Generate the query instance using OpenAI
    query_instance = await generate_query_instance()

    # Call the execute method and print the results
    result = await query_instance.execute()
    print(result)

# Run the main function using nest_asyncio to allow nesting the event loop
await main()

{'results': [{'title': "What a long, strange year it's been in enterprise tech news", 'url': 'https://techcrunch.com/2023/12/31/enterprise-tech-generative-ai-2023/', 'author': 'Ron Miller', 'id': 'https://techcrunch.com/2023/12/31/enterprise-tech-generative-ai-2023/', 'text': 'From Salesforce drama to the year of generative AI\n \n Grateful Dead, but what a long, strange year it’s been in 2023 enterprise tech news. It began with a ton of Salesforce drama and eventually got taken over by generative AI and ChatGPT, which seemed to come out of nowhere to completely dominate the news cycle this year.\nBut even though AI clearly influenced much of the news, and even my own coverage, there was still a ton of other enterprise stories that made the news this year.\nThe rise of generative AI in the enterprise\nIt would be impossible to discuss this year’s news cycle without talking about the impact of generative AI. When OpenAI released ChatGPT at the end of last year, it would have been imposs